In [ ]:
from simulator_interface import open_session, close_session
simulator, epuck1, epuck2 = open_session(n_epucks=2)

In [ ]:
close_session(simulator)

In [ ]:
def obstacle_avoidance(robot):
    left, right = robot.prox_activations(tracked_objects=["20cm"])
    left_wheel = 1 - right
    right_wheel = 1 - left   
    return left_wheel, right_wheel

In [ ]:
def safe_place_avoidance(robot): 
    left, right = robot.prox_activations(tracked_objects=["Cup", "Tree"])
    left_wheel = left
    right_wheel = right
    return left_wheel,  right_wheel, robot.energy_level

In [ ]:
def towards_safe_place(robot): 
    left, right = robot.prox_activations(tracked_objects=["Tree"])
    left_wheel = 1 - left
    right_wheel = 1 - right
    return left_wheel, right_wheel, robot.energy_level

In [ ]:
left, right = epuck1.prox_activations(tracked_objects=["ePuck"])
print(left, right)

In [ ]:
simulator.start_sphere_apparition(period=20., min_pos=[-1.3, 0.66, 0], max_pos=[-1.1, 0.68, 1])

In [ ]:
def foraging (robot):
    left, right = robot.prox_activations(tracked_objects=["Sphere"])
    return right, left  

In [ ]:
def energy_drive (robot): 
    if robot.has_eaten():
        robot.energy_level += 0.2  # if the robot has eaten a sphere, increase its energy level by 0.2
    else:
        robot.energy_level -= 0.01  # otherwise (nothing eaten), decrease the energy level by 0.01
    # The line below bounds the value of the energy level between 0 and 1
    robot.energy_level = min(1., max(robot.energy_level, 0.))

In [ ]:
epuck1.danger = 0
epuck2.danger = 0

def alarm_drive (robot):
    left, right = robot.prox_activations(tracked_objects=["ePuck"])
    if left + right > 0:
        robot.danger = left + right 
    else:
        robot.danger = 0
    # The line below bounds the value of the robot's danger between 0 and 1
    robot.danger = min(1., max(robot.danger, 0.))

In [ ]:
def epuck_log(robot):
    # Retrieve the values of the left and right proximeters:
    left, right = robot.prox_activations()
    
    # record the energy level in the topic called "energy"
    robot.add_log("energy", robot.energy_level)

In [ ]:
def escaping (robot):
    if robot.danger > 0:
        left, right = robot.prox_activations(tracked_objects=["ePuck"])
        return left, right
    else:
        left, right = robot.prox_activations(tracked_objects=["Cup"])
        return right, left, robot.energy_level

In [ ]:
def chasing (robot):
    left, right = robot.prox_activations(tracked_objects=["ePuck"])
    return right, left, robot.energy_level

In [ ]:
for e in simulator.robots:
    e.detach_all_behaviors()
    e.attach_behavior(obstacle_avoidance, freq=10)
    e.attach_behavior(foraging, freq=10)
    e.attach_routine(alarm_drive, freq=1)
    e.attach_routine(energy_drive, freq=1)
    e.attach_routine(epuck_log, freq=1)
    
    e.energy_level = 0.5

epuck1.attach_behavior(chasing, freq=10)
epuck1.attach_behavior(safe_place_avoidance, freq=10)
    
epuck2.attach_behavior(escaping, freq=10)
epuck2.attach_behavior(towards_safe_place, freq=10)



for e in simulator.robots:
    e.start_all_behaviors()
    e.start_all_routines ()

In [ ]:
print(epuck1.energy_level)
print(epuck2.energy_level)

In [ ]:
# The line below is mandatory to inform the notebook we want to plot directly in it
%pylab inline

# Plot the energy levels recorded by `epuck1` and `epuck2`
plot(epuck1.get_log("energy"))
plot(epuck2.get_log("energy"))
legend(["Police", "Thief"])

xlabel("Time")
ylabel("Energy level")
title("Plot of energy level against time")